# Designing a Jupyter Extensions with IPyDrawio

Diagrams can effectively support the many phases of the software development cycle. What IPyDrawio, and the underlying general-purpose drawio UI, lack in features versus more specialized tools, when used in the right context to help a software team communicate between itself, and sometimes more importantly, its users.

```{note}
This tutorial looks at how different kinds of diagram artifacts can be created that support the process of building _Jupyter Extensions_, using IPyDrawio itself as a subject. Each of the images below rendered directly in the browser for this documentation, as well as editable within IPyDrawio.
```

Extensions for [JupyterLab](https://jupyterlab.readthedocs.io/en/stable/extension/extension_dev.html) and [Jupyter Server](https://jupyter-server.readthedocs.io/en/latest/developers/extensions.html?highlight=distributing#distributing-a-server-extension) often developed, and can be distributed, together as a single Python package. While a single author can build, test, document and ship new features _without_ any kinds of pictures, individuals and especially team- and community-driven processes can be enhanced by different visual artifacts that _live next to the code_.

```{hint}
Using an editable, but widely-viewable format like `.dio.png` or `.dio.svg` means these assets can be used "at-rest," though without advanced features like multiple pages and interactivity, and works well with the visual diff tools on e.g. GitHub.
```

## Gathering Ideas

As you start thinking about your extension, who will use it, and how you might build it, IPyDrawio diagrams can capture structured problem-solving approaches effective at capturing, and then leveraging, visual concepts to capture goals and trade-offs.

### Mind Maps
[Mind Maps](https://en.wikipedia.org/wiki/Mind_map) are an effective way to capture "stream of consciousness".

```{hint}
Search for `mind` in the _Custom Diagram..._ panel. This task also lends itself to the `sketch` theme, which trades UI simplicity for hiding some of the more advanced features.
```

![a mind map of ipydrawio](../../_static/tutorials/designing-jupyter-extensions/mindmap.dio.svg)

### Seven Management and Planning Tools
The [Seven Management and Planning Tools](https://en.wikipedia.org/wiki/Seven_management_and_planning_tools) are suitable for large and small projects.

```{note}
While there are no "prebuilt" templates for these diagrams, many of the "basic" and and "tree" shape palettes have ready-made shapes with pre-configured connectors that preserve intent, and work well with the auto-layout tools.
```

![an affinity diagram of ipydrawio](../../_static/tutorials/designing-jupyter-extensions/affinity.dio.svg)

```{hint}
Use the <kbd>Ctrl</kbd> (or <kbd>Cmd</kbd> on MacOS) with existing arrows to quickly make copies.  
```

### Mermaid Diagrams

The [mermaid.js](https://mermaid-js.github.io/mermaid) drawio plugin (enabled by default) provides a way to write pithy text descriptions, including:

> _Flowchart, Sequence diagram, Class Diagram, State Diagram, Entity Relationship Diagram, User Journey, Gantt, Pie Chart, Requirement Diagram_

```{note}
From the drawio menu, select _Insert_ ▸ _Advanced_ ▸ _Mermaid_. Paste some mermaid there. The resulting object will _not_ be editable with shape-editing tools, but you _will_ be able to adjust the mermaid syntax later by clicking on it, which can be worth _much_ more over time. 
```

```{hint}
The [mermaid live editor](https://mermaid-js.github.io/mermaid-live-editor) is a more interactive editing experience than the `textarea` provided by drawio.
```

![an affinity diagram of ipydrawio](../../_static/tutorials/designing-jupyter-extensions/mermaid.dio.svg)

If these kinds of diagrams are helpful for your extension, consider more advanced tools like [Papyrus](https://www.eclipse.org/papyrus) that support more formal models and diagram standards.

## Visual Design

While IPyDrawio isn't as full-featured as properietary tools, or even open source tools such as the excellent [Inkscape](https://inkscape.org), it can be effective, and in some ways, more approachable, than full design suites.

### Wireframing

Low-fidelity wireframes, which often include filler text like _lorem ipsum_, placeholder images, etc. provide a quick way to capture the essential views of a novel UI without getting a team hung up on details.

![a wireframe of the ipydrawio custom diagram panel](../../_static/tutorials/designing-jupyter-extensions/wireframe.dio.svg)

```{hint}
Use primarily _basic shapes_: a box with a word in it says "button" just as well as something with gradients and carefully-kerned fonts.
```

```{note}
For single-page/stage views, the `sketch` theme can work well, while `min` provides a nice balance between unobtrusiveness and power.
```

### Comps

High-fidelity mockups, or "comps," eventually provide the language used between the designer role and the developer role... even if these roles are filled by the same person.

```{hint}
Search for `jupyter` in the _Custom Diagram..._ panel. The _JupyterLab Mockups_ contains recreations of a number of common views, such as the _Launcher_ and _Notebook_, which can be recombined, remixed, or expanded to suggest new features. 
```

```{note}
One of the "full" themes (e.g. `kennedy`, `dark` or `atlas`) is recommended, as using multiple pages, cross-page links, and auto-layouts are more readily discoverable than with `min` or `sketch`.
```

![a comp of a future feature for IPyDrawio](../../_static/tutorials/designing-jupyter-extensions/comp.dio.svg)

## Testing

While not a _particularly_ accessible semantic model, native drawio documents and SVG _are_ well-behaved XML, and far less opaque than "professional" UML/SysML tools. Especially when using some of the richer shapes, such as those in _Trees_, some data will be preserved, and can be used, for example, to validate repository integrity.


```{note}
For example, if you are building an extension composed of many classes, you can use either simple string comparison, regular expressions, or XPath queries (provided by both [browser JS](https://developer.mozilla.org/en-US/docs/Web/XPath) and [python](https://docs.python.org/3/library/xml.etree.elementtree.html#elementtree-xpath)) to verify all classes are included in a key diagram.
```

![a deployment diagram of ipydrawio packages](../../_static/tutorials/designing-jupyter-extensions/deployment.dio.svg)

In [1]:
import lxml.etree as ET
from pathlib import Path

Because the sources and targets of the dotted edges are not preserved on the `path` SVG elements, we use the fill colors to fill in the missing data.

In [2]:
packages = {"#dae8fc": "ipydrawio", "#f8cecc": "ipydrawio_export"}

In [3]:
ships = {
    py_pkg: {
        js_pkg["dest"]
        for js_pkg
        in __import__(py_pkg)._jupyter_labextension_paths()
    }
    for fill, py_pkg in packages.items()
}

In [4]:
def test_deployment(
    ns="@deathbeds/",
    diagram="../../_static/tutorials/designing-jupyter-extensions/deployment.dio.svg"
):
    deployment = Path(diagram)
    et = ET.fromstring(deployment.read_text(encoding="utf-8"))
    # get everything with a data-label, which gets preserved
    should_ship_in = {
        f"""{ns}{e.attrib["data-label"]}""":
            packages.get(e.xpath("./*[1]/@fill")[0])
        for e in et.xpath("//*[@data-label]")
    }
    for py_package, ships_exts in ships.items():
        print(py_package)
        print("... observed", sorted(ships_exts))
        should_ship_exts = {
            js_name for js_name, py_name in should_ship_in.items()
            if py_name == py_package
        }
        print("... expected", sorted(should_ship_exts))
        mismatches = ships_exts ^ should_ship_exts
        assert not mismatches, f"mismatches: {mismatches}"
        print(f"... {py_package} is 🚢-shape!")

```{note}
For documentation purposes, we just run this directly here, but this could be easily run as part of a regular unit test suite, and an "offending" diagram included in a rich test output such as [pytest-html](https://github.com/pytest-dev/pytest-html).
```

In [5]:
test_deployment()

ipydrawio
... observed ['@deathbeds/ipydrawio', '@deathbeds/ipydrawio-jupyter-templates', '@deathbeds/ipydrawio-notebook', '@deathbeds/ipydrawio-webpack']
... expected ['@deathbeds/ipydrawio', '@deathbeds/ipydrawio-jupyter-templates', '@deathbeds/ipydrawio-notebook', '@deathbeds/ipydrawio-webpack']
... ipydrawio is 🚢-shape!
ipydrawio_export
... observed ['@deathbeds/ipydrawio-pdf']
... expected ['@deathbeds/ipydrawio-pdf']
... ipydrawio_export is 🚢-shape!


Now, if a new package is added to either the code, or the diagram, and the other side is _not_ updated, this test will fail, helping to keep the documentation up-to-date.

## Documentation

Once these diagrams exist, they can be used directly within documentation sites. While alternatives exist for working directly with `.drawio` files, such as [sphinxcontrib-drawio](https://pypi.org/project/sphinxcontrib-drawio), again keeping the files in standards-compliant formats such as `.png` and `.svg` are more likely to stay editable and viewable over time.

```{hint}
[myst-nb](https://github.com/executablebooks/MyST-NB) is highly recommended (and used on this documentation site) for embedding various kinds of images without a lot of the hassle of some of the finer points of `.rst` editing.
```

Another approach is generating diagrams, with tools such as [graphviz2drawio](https://github.com/hbmartin/graphviz2drawio) or [n2g](https://github.com/dmulyalin/N2G). Both offer an intermediate representation which can be more effective at keeping documentation up-to-date than hand editing diagrams.